In [ ]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from pathlib import Path
pd.set_option('display.max_rows', 25990)
pd.set_option('display.max_columns', 200)

In [ ]:
# Define the file path
BASE = Path("/content/drive/MyDrive/CP_UMBC/Raw_Data")
FILE = BASE / "RT2733808 Fin aid stdnt Success analysis data Sp2025.xlsx"

xls = pd.ExcelFile(FILE)
print("Available sheets:", xls.sheet_names)


Available sheets: ['FA Stdnt Success ', 'AP Courses']


In [ ]:
# Load both sheets
sheet1 = pd.read_excel(FILE, sheet_name=xls.sheet_names[0])

In [ ]:
sheet1.columns

Index(['StudentKey', 'EmployeeID', 'MatricTermdescription',
       'MatricAcademicYear', 'MatricStatusOfficialDescr', 'MatricGenderIPEDS',
       'MatricIPEDSEthnicity', 'Zipcode', 'MatricResidence', 'Graduated',
       ...
       'HS_PecentileDesc', 'HasAlgScore', 'AlgSCORE', 'HasCalScore',
       'CalScore', 'HasALEKSscore', 'ALEKSScore', 'HasEngScore', 'EngSCORE',
       'AP_CRDS'],
      dtype='object', length=131)

In [ ]:
sheet1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25960 entries, 0 to 25959
Columns: 131 entries, StudentKey to AP_CRDS
dtypes: float64(54), int64(18), object(59)
memory usage: 25.9+ MB


In [ ]:
import numpy as np
sheet1 = sheet1.copy()

In [ ]:
# Define yearly column groups
need_cols  = [f"Y{i}Need" for i in range(1, 7)]
grant_cols = [f"Y{i}GrantAmount" for i in range(1, 7)]
merit_cols = [f"Y{i}MeritAmount" for i in range(1, 7)]

# Total Support = sum of all grant + merit amounts (Y1–Y6)
sheet1["TotalSupport"] = sheet1[grant_cols + merit_cols].sum(axis=1, skipna=True)

#  Supported flag (Yes/No)
sheet1["Supported"] = np.where(sheet1["TotalSupport"] > 0, "Yes", "No")

#  NeedStatus (Yes/No if any year shows real financial need)
sheet1["NeedStatus"] = np.where(
    sheet1[need_cols].apply(
        lambda row: any(
            (isinstance(val, str) and val.strip().lower() not in ["no fafsa", "unknown", "none", ""])
            for val in row
        ),
        axis=1
    ),
    "Yes",
    "No"
)

#   Bucket TotalSupport into ranges
bins = [-0.1, 0, 5000, 10000, 15000, 20000, np.inf]
labels = ["NoSup", "<5K", "5K-10K", "10K-15K", "15K-20K", ">20K"]
sheet1["SupportBin"] = pd.cut(sheet1["TotalSupport"], bins=bins, labels=labels)

print(sheet1[["EmployeeID", "TotalSupport", "Supported", "NeedStatus", "SupportBin"]].head(10))


   EmployeeID  TotalSupport Supported NeedStatus SupportBin
0  2000058476           0.0        No        Yes      NoSup
1  2000067360           0.0        No         No      NoSup
2  2000077103        5000.0       Yes         No        <5K
3  2000107627           0.0        No         No      NoSup
4  2000037534       39711.0       Yes        Yes       >20K
5  2000037570       21015.0       Yes        Yes       >20K
6  2000037804           0.0        No        Yes      NoSup
7  2000059165           0.0        No        Yes      NoSup
8  1000011230           0.0        No        Yes      NoSup
9  1000011462        1500.0       Yes        Yes        <5K


In [ ]:
support_bin_stats = (
    sheet1.groupby("SupportBin")
    .agg(
        Count=("SupportBin", "size"),
        Graduated_Count=("Graduated", lambda x: (x == "Yes").sum()),
        Graduation_Rate=("Graduated", lambda x: (x == "Yes").mean())
    )
    .sort_index()
)

print(support_bin_stats)


            Count  Graduated_Count  Graduation_Rate
SupportBin                                         
NoSup        8485             4237         0.499352
<5K          4106             1164         0.283488
5K-10K       2385              781         0.327463
10K-15K      2118              874         0.412653
15K-20K      1538              939         0.610533
>20K         7328             4860         0.663210


/tmp/ipython-input-2531406609.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  sheet1.groupby("SupportBin")


In [ ]:
# Filter rows where HighSchoolGpa is missing
no_gpa = sheet1[sheet1["HighSchoolGpa"].isna()].copy()

print("No GPA student count:", no_gpa.shape[0])
no_gpa.head()


No GPA student count: 7870


,StudentKey,EmployeeID,MatricTermdescription,MatricAcademicYear,MatricStatusOfficialDescr,MatricGenderIPEDS,MatricIPEDSEthnicity,Zipcode,MatricResidence,Graduated,Yrs_To_Deg,Sem_To_Deg,Plan1_1,Plan1_2,Plan1_3,Plan1_4,Plan1_5,Plan1_6,Plan1_7,Plan1_8,Plan1_9,Plan1_10,Plan1_11,Plan1_12,Plan1_13,Plan1_14,Plan1_15,Plan1_16,Deg_8,Deg_10,Deg_12,LastSTEMType,LastrptgPlanShortDescr,Sem1_FTPT,Sem2_FTPT,Sem3_FTPT,Sem4_FTPT,Sem5_FTPT,Sem6_FTPT,Sem7_FTPT,Sem8_FTPT,Sem9_FTPT,Sem10_FTPT,Sem11_FTPT,Sem12_FTPT,Sem13_FTPT,Sem14_FTPT,Sem15_FTPT,Sem16_FTPT,Same_STEM_1_2,Same_STEM_1_21,Same_STEM_1_3,Same_STEM_1_4,Same_STEM_1_5,Same_STEM_1_6,Same_STEM_1_7,Same_STEM_1_8,Same_STEM_1_9,Same_STEM_1_10,Same_STEM_1_11,Same_STEM_1_12,Same_STEM_1_13,Same_STEM_1_14,Same_STEM_1_15,Same_STEM_1_16,Degr4Sem,Degr5Sem,Degr6Sem,Degr7Sem,Degr8Sem,Degr9Sem,Degr10Sem,Degr11Sem,Degr12Sem,Degr13Sem,Degr14Sem,Degr15Sem,Degr16Sem,Y1Need,Y2Need,Y3Need,Y4Need,Y5Need,Y6Need,Y1GrantAmount,Y2GrantAmount,Y3GrantAmount,Y4GrantAmount,Y5GrantAmount,Y6GrantAmount,Y1MeritAmount,Y2MeritAmount,Y3MeritAmount,Y4MeritAmount,Y5MeritAmount,Y6MeritAmount,Y1ScholarProgram,Y1ProgramAmount,Y2ScholarProgram,Y2ProgramAmount,Y3ScholarProgram,Y3ProgramAmount,Y4ScholarProgram,Y4ProgramAmount,Y5ScholarProgram,Y5ProgramAmount,Y6ScholarProgram,Y6ProgramAmount,MatricResidencyTuitionDescript,HighSchoolDescription,HighSchoolState,HighSchoolCounty,HighSchoolGpa,HighSchoolGPABandDescription,HasSATMathScore,SAT1600Score,SATMathScore,HasSATReadingWritingScore,SATReadingWritingScore,HasHighSchoolRankPercentile,HighSchoolWeightedRankPercentile,HS_PecentileDesc,HasAlgScore,AlgSCORE,HasCalScore,CalScore,HasALEKSscore,ALEKSScore,HasEngScore,EngSCORE,AP_CRDS,TotalSupport,Supported,NeedStatus,SupportBin
2,315405,2000077103,Fall 2012,2013,New Transfer,Male,Asian,20740,Commuter,No,NaN,NaN,Biol Sci,Biol Sci,Biol Sci,Biol Sci,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Undecided,NaN,NaN,NaN,NaN,0.0,0.0,0.0,Not STEM Plan,Undecided,FT,FT,FT,FT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PT,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,No Fafsa,No Fafsa,No Fafsa,No Fafsa,No Fafsa,No Fafsa,0.0,0.0,0.0,0.0,0.0,0.0,2500.0,2500.0,0.0,0.0,0.0,0.0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,In State Resident,Rockville High School ...,MD,Montgomery,NaN,Unknown,0,NaN,NaN,0,NaN,0,0,Unknown,0,NaN,0,NaN,0,NaN,0,NaN,0,5000.0,Yes,No,<5K
3,319368,2000107627,Spring 2013,2013,New Transfer,Male,White,20850,On Campus,Yes,1.5,3.0,Biol Sci,Biol Sci,Biol Sci,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,STEM Plan,Biol Sci,FT,FT,FT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,No Fafsa,No Fafsa,No Fafsa,No Fafsa,No Fafsa,No Fafsa,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,In State Resident,Thomas Sprigg Wootton Hs ...,MD,Montgomery,NaN,Unknown,0,NaN,NaN,0,NaN,0,0,Unknown,0,NaN,0,NaN,0,NaN,0,NaN,34,0.0,No,No,NoSup
4,334400,2000037534,Fall 2015,2016,New Transfer,Male,Asian,21228,Commuter,Yes,4.0,8.0,Biol Educ,Biol Educ,Biol Educ,Biol Educ,Biol Educ,Biol Educ,Biol Educ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,STEM Plan,Biol Educ,FT,FT,FT,FT,FT,FT,PT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Low,High,High,High,High,No Fafsa,0.0,5815.0,13460.0,4186.0,0.0,0.0,5000.0,5000.0,5000.0,1250.0,0.0,0.0,NaN,0,Sherman,5000,Sherman,5000,Sherman,1250,NaN,0,NaN,0,In State Resident,Catonsville High School ...,MD,Baltimore County,NaN,Unknown,0,NaN,NaN,0,NaN,0,0,Unknown,1,26.0,1,1.0,0,NaN,0,NaN,3,39711.0,Yes,Yes,>20K
5,334435,2000037570,Spring 2015,2015,New Transfer,Female,White,21036,Commuter,Yes,3.0,6.0,Comp Sci,Comp Sci,Comp Sci,Comp Sci,Comp Sci,Comp Sci,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,STEM Plan,Comp Sc

In [ ]:
no_gpa_grad = (
    no_gpa["Graduated"]
    .value_counts(dropna=False)
    .rename({"Yes":"Graduated", "No":"Not Graduated"})
)

no_gpa_rate = (no_gpa["Graduated"] == "Yes").mean()

print(no_gpa_grad)
print(f"\nGraduation rate for No GPA students: {no_gpa_rate:.3%}")


Graduated
Graduated        4206
Not Graduated    3664
Name: count, dtype: int64

Graduation rate for No GPA students: 53.443%


In [ ]:
transfer_check = (
    no_gpa.groupby("MatricStatusOfficialDescr")["Graduated"]
    .apply(lambda x: (x=="Yes").mean())
    .to_frame("Graduation_Rate")
)

print("\nTransfer vs Freshman in No GPA group:")
print(transfer_check)

print("\nCounts:")
print(no_gpa["MatricStatusOfficialDescr"].value_counts())



Transfer vs Freshman in No GPA group:
                           Graduation_Rate
MatricStatusOfficialDescr                 
New Freshman                      0.448718
New Transfer                      0.536168

Counts:
MatricStatusOfficialDescr
New Transfer    7714
New Freshman     156
Name: count, dtype: int64


In [ ]:
need_check = (
    no_gpa["Y1Need"]
    .astype(str)
    .str.strip()
    .str.lower()
    .value_counts()
)

print("\nY1 Need distribution in No GPA group:")
print(need_check)



Y1 Need distribution in No GPA group:
Y1Need
high        3198
no fafsa    2494
low         1292
medium       886
Name: count, dtype: int64


In [ ]:
need_status_check = (
    no_gpa.groupby("NeedStatus")["Graduated"]
    .apply(lambda x: (x=="Yes").mean())
)

print("\nGraduation rate by NeedStatus (No GPA group):")
print(need_status_check)

print("\nCounts:")
print(no_gpa["NeedStatus"].value_counts())



Graduation rate by NeedStatus (No GPA group):
NeedStatus
No     0.502713
Yes    0.545439
Name: Graduated, dtype: float64

Counts:
NeedStatus
Yes    5843
No     2027
Name: count, dtype: int64


In [ ]:
residence_check = (
    no_gpa.groupby("MatricResidence")["Graduated"]
    .apply(lambda x: (x=="Yes").mean())
)

print("\nResidence impact in No GPA group:")
print(residence_check)

print("\nCounts:")
print(no_gpa["MatricResidence"].value_counts())



Residence impact in No GPA group:
MatricResidence
Commuter     0.525616
On Campus    0.588879
Name: Graduated, dtype: float64

Counts:
MatricResidence
Commuter     6773
On Campus    1097
Name: count, dtype: int64


In [ ]:
placement_cols = ["AlgSCORE", "CalScore", "ALEKSScore", "EngSCORE"]

print("\nPlacement score statistics for No GPA students:")
print(no_gpa[placement_cols].describe())



Placement score statistics for No GPA students:
         AlgSCORE    CalScore   ALEKSScore    EngSCORE
count  618.000000  619.000000  1149.000000  660.000000
mean    22.920712    9.730210    67.877285    4.521212
std      6.822253    9.443748    21.246637    1.608354
min      4.000000    1.000000     3.000000    3.000000
25%     18.000000    1.000000    53.000000    4.000000
50%     24.000000    1.000000    74.000000    4.000000
75%     29.000000   19.500000    84.000000    4.000000
max     32.000000   25.000000    99.000000    9.000000


In [ ]:
major_check = (
    no_gpa["Plan1_1"]
    .value_counts()
    .head(20)
)

print("\nTop majors for No GPA students:")
print(major_check)



Top majors for No GPA students:
Plan1_1
Biol Sci                  1696
Pre-Comp Sci              1396
Info Sys                  1119
Bus Tech Admin             476
Bioc & Mol Biol            402
Pre-Mech Eng               377
Comp Sci                   370
Mathematics                263
Pre-Comp Eng               211
Mech Eng                   184
Pre-Chem Eng               163
Pre-Engineering            152
Chemistry                  151
Physics                    136
Chemical Eng               125
Environ Sci & Geog         114
Comp Eng                   110
Environ Sci                 76
Trnsltnl Life Sci Tech      72
Engineering                 68
Name: count, dtype: int64


NO SUPPORT


In [ ]:
no_sup = sheet1[sheet1["SupportBin"] == "NoSup"].copy()

print("No Support student count:", no_sup.shape[0])
no_sup.head()


No Support student count: 8485


,StudentKey,EmployeeID,MatricTermdescription,MatricAcademicYear,MatricStatusOfficialDescr,MatricGenderIPEDS,MatricIPEDSEthnicity,Zipcode,MatricResidence,Graduated,Yrs_To_Deg,Sem_To_Deg,Plan1_1,Plan1_2,Plan1_3,Plan1_4,Plan1_5,Plan1_6,Plan1_7,Plan1_8,Plan1_9,Plan1_10,Plan1_11,Plan1_12,Plan1_13,Plan1_14,Plan1_15,Plan1_16,Deg_8,Deg_10,Deg_12,LastSTEMType,LastrptgPlanShortDescr,Sem1_FTPT,Sem2_FTPT,Sem3_FTPT,Sem4_FTPT,Sem5_FTPT,Sem6_FTPT,Sem7_FTPT,Sem8_FTPT,Sem9_FTPT,Sem10_FTPT,Sem11_FTPT,Sem12_FTPT,Sem13_FTPT,Sem14_FTPT,Sem15_FTPT,Sem16_FTPT,Same_STEM_1_2,Same_STEM_1_21,Same_STEM_1_3,Same_STEM_1_4,Same_STEM_1_5,Same_STEM_1_6,Same_STEM_1_7,Same_STEM_1_8,Same_STEM_1_9,Same_STEM_1_10,Same_STEM_1_11,Same_STEM_1_12,Same_STEM_1_13,Same_STEM_1_14,Same_STEM_1_15,Same_STEM_1_16,Degr4Sem,Degr5Sem,Degr6Sem,Degr7Sem,Degr8Sem,Degr9Sem,Degr10Sem,Degr11Sem,Degr12Sem,Degr13Sem,Degr14Sem,Degr15Sem,Degr16Sem,Y1Need,Y2Need,Y3Need,Y4Need,Y5Need,Y6Need,Y1GrantAmount,Y2GrantAmount,Y3GrantAmount,Y4GrantAmount,Y5GrantAmount,Y6GrantAmount,Y1MeritAmount,Y2MeritAmount,Y3MeritAmount,Y4MeritAmount,Y5MeritAmount,Y6MeritAmount,Y1ScholarProgram,Y1ProgramAmount,Y2ScholarProgram,Y2ProgramAmount,Y3ScholarProgram,Y3ProgramAmount,Y4ScholarProgram,Y4ProgramAmount,Y5ScholarProgram,Y5ProgramAmount,Y6ScholarProgram,Y6ProgramAmount,MatricResidencyTuitionDescript,HighSchoolDescription,HighSchoolState,HighSchoolCounty,HighSchoolGpa,HighSchoolGPABandDescription,HasSATMathScore,SAT1600Score,SATMathScore,HasSATReadingWritingScore,SATReadingWritingScore,HasHighSchoolRankPercentile,HighSchoolWeightedRankPercentile,HS_PecentileDesc,HasAlgScore,AlgSCORE,HasCalScore,CalScore,HasALEKSscore,ALEKSScore,HasEngScore,EngSCORE,AP_CRDS,TotalSupport,Supported,NeedStatus,SupportBin
0,8842,2000058476,Fall 2021,2022,New Transfer,Female,White,21228,Commuter,No,NaN,NaN,Pre-Mech Eng,Pre-Mech Eng,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,STEM Plan,Pre-Mech Eng,FT,PT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,High,Medium,No Fafsa,No Fafsa,No Fafsa,No Fafsa,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,In State Resident,Atholton High School ...,MD,Howard,4.29,4.0 - 4.49,0,NaN,NaN,0,NaN,0,0,Unknown,0,NaN,0,NaN,0,NaN,0,NaN,0,0.0,No,Yes,NoSup
1,9141,2000067360,Fall 2019,2020,New Transfer,Female,Not Specified,21201,Commuter,No,NaN,NaN,Info Sys,Info Sys,Info Sys,Info Sys,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,STEM Plan,Info Sys,PT,PT,PT,PT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,No Fafsa,No Fafsa,No Fafsa,No Fafsa,No Fafsa,No Fafsa,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,In State Resident,Catonsville High School ...,MD,Baltimore County,3.68,3.5 - 3.99,0,NaN,NaN,0,NaN,0,0,Unknown,0,NaN,0,NaN,0,NaN,0,NaN,0,0.0,No,No,NoSup
3,319368,2000107627,Spring 2013,2013,New Transfer,Male,White,20850,On Campus,Yes,1.5,3.0,Biol Sci,Biol Sci,Biol Sci,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,STEM Plan,Biol Sci,FT,FT,FT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,No Fafsa,No Fafsa,No Fafsa,No Fafsa,No Fafsa,No Fafsa,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,In State Resident,Thomas Sprigg Wootton Hs ...,MD,Montgomery,NaN,Unknown,0,NaN,NaN,0,NaN,0,0,Unknown,0,NaN,0,NaN,0,NaN,0,NaN,34,0.0,No,No,NoSup
6,332143,2000037804,Fall 2013,2014,New Transfer,Male,Hispanic/Latino,21076,Commuter,Yes,7.5,15.0,Comp Sci,Comp Sci,Comp Sci,Comp Sci,NaN,NaN,NaN,NaN,Comp Sci,Comp Sci,Comp Sci,Comp Sci,Comp Sci,Comp Sci,Comp Sci,NaN,0.0,0.0,0.0,STEM Plan,Comp Sci,FT,FT,PT,FT,NaN,NaN,NaN

In [ ]:
no_sup_grad = (
    no_sup["Graduated"]
    .value_counts(dropna=False)
    .rename({"Yes":"Graduated", "No":"Not Graduated"})
)

no_sup_rate = (no_sup["Graduated"] == "Yes").mean()

print(no_sup_grad)
print(f"\nGraduation rate for No Support students: {no_sup_rate:.3%}")


Graduated
Not Graduated    4248
Graduated        4237
Name: count, dtype: int64

Graduation rate for No Support students: 49.935%


In [ ]:
transfer_sup = (
    no_sup.groupby("MatricStatusOfficialDescr")["Graduated"]
    .apply(lambda x: (x=="Yes").mean())
)

print("\nGraduation rate by MatricStatus within No Support:")
print(transfer_sup)

print("\nCounts:")
print(no_sup["MatricStatusOfficialDescr"].value_counts())



Graduation rate by MatricStatus within No Support:
MatricStatusOfficialDescr
New Freshman    0.518825
New Transfer    0.482115
Name: Graduated, dtype: float64

Counts:
MatricStatusOfficialDescr
New Transfer    4501
New Freshman    3984
Name: count, dtype: int64


In [ ]:
nosup_need = (
    no_sup["Y1Need"]
    .astype(str).str.strip().str.lower()
    .value_counts()
)

print("\nY1 Need distribution in No Support group:")
print(nosup_need)



Y1 Need distribution in No Support group:
Y1Need
no fafsa    3922
low         3152
medium      1122
high         289
Name: count, dtype: int64


In [ ]:
nosup_needstatus = (
    no_sup.groupby("NeedStatus")["Graduated"]
    .apply(lambda x: (x=="Yes").mean())
)

print("\nGraduation rate by NeedStatus (No Support):")
print(nosup_needstatus)

print("\nCounts:")
print(no_sup["NeedStatus"].value_counts())



Graduation rate by NeedStatus (No Support):
NeedStatus
No     0.502477
Yes    0.497230
Name: Graduated, dtype: float64

Counts:
NeedStatus
Yes    5054
No     3431
Name: count, dtype: int64


In [ ]:
nosup_residence = (
    no_sup.groupby("MatricResidence")["Graduated"]
    .apply(lambda x: (x=="Yes").mean())
)

print("\nResidence impact (No Support):")
print(nosup_residence)

print("\nCounts:")
print(no_sup["MatricResidence"].value_counts())



Residence impact (No Support):
MatricResidence
Commuter     0.460167
On Campus    0.563648
Name: Graduated, dtype: float64

Counts:
MatricResidence
Commuter     5272
On Campus    3213
Name: count, dtype: int64


In [ ]:
placement_cols = ["AlgSCORE", "CalScore", "ALEKSScore", "EngSCORE"]

print("\nPlacement scores for No Support:")
print(no_sup[placement_cols].describe())



Placement scores for No Support:
          AlgSCORE     CalScore   ALEKSScore     EngSCORE
count  2294.000000  2295.000000  2368.000000  2886.000000
mean     25.355711    14.318519    69.065878     4.126473
std       5.155402     9.026073    19.429199     1.043730
min       1.000000     1.000000     2.000000     3.000000
25%      23.000000     1.000000    57.000000     4.000000
50%      26.000000    19.000000    75.000000     4.000000
75%      29.000000    22.000000    83.000000     4.000000
max      32.000000    25.000000   100.000000     9.000000


In [ ]:
nosup_major = (
    no_sup["Plan1_1"]
    .value_counts()
    .head(15)
)

print("\nTop majors for No Support:")
print(nosup_major)



Top majors for No Support:
Plan1_1
Biol Sci           1845
Pre-Comp Sci       1555
Info Sys            766
Comp Sci            490
Pre-Mech Eng        482
Bus Tech Admin      469
Bioc & Mol Biol     435
Pre-Engineering     330
Pre-Comp Eng        306
Mech Eng            243
Mathematics         242
Engineering         187
Pre-Chem Eng        164
Chemistry           151
Comp Eng            144
Name: count, dtype: int64
